## reading in all tara data

 # marker–metatranscriptome correlation analysis
    
**Purpose:**  
    
    This notebook takes the cleaned counts of functional marker genes (e.g. *nod*, aerobic- and anaerobic-metabolism genes) from the Tara Oceans metatranscriptome data set, isolates the low-O₂ sample subset, and explores the relationship between *nod* expression and an `aerobic_score` proxy.

    The key steps kept in this trimmed version are:
    1. **Scatter plots** of *nod* CPM versus the aerobic score for all low-oxygen samples, with a fitted trendline for quick visual assessment.
    2. **Spearman rank correlation** calculation (ρ) to quantify the monotonic association between the two variables.
    3. **Non-parametric bootstrap** (100 000 resamples) to obtain a robust 95 % confidence interval for ρ, helping gauge the reliability of the observed correlation.

    All upstream data-loading and downstream exploratory cells have been removed so you can rerun just the essential statistical segment quickly. Execute the notebook top-to-bottom and the final cell prints the correlation with its bootstrap CI.


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.patches as mpatches
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
from scipy.stats import pearsonr


tara_sum = pd.read_csv(r"/Users/theogoodchild-michelman/MIT Dropbox/Izzy Goodchild-Michelman/Dropbox_files/20.440/final_tara_caricao_output/all_tara_sum_try3_CPM.csv")
caricao_sum = pd.read_csv(r"/Users/theogoodchild-michelman/MIT Dropbox/Izzy Goodchild-Michelman/Dropbox_files/20.440/final_tara_caricao_output/all_caricao_counts_sum2.csv")


caricao_sum['Cyt_bd_oxid'] = caricao_sum[['Cyt_bd_oxida_I', 'cytbd']].sum(axis=1)
caricao_sum = caricao_sum.drop(columns=['Cyt_bd_oxida_I', 'cytbd'], axis=1)



/Users/theogoodchild-michelman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/theogoodchild-michelman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
tara_sum

,sample,cox1,cox3,dioxygenase,dsr,katG,nap,narG,nir,nod,nos,nrd,pmoA,ribo_reductase,sod,Cyt_bd_oxid
0,TARA_A100000165,377.412099,198.565004,0.118061,41.444740,285.772373,1.061463,0.000000,0.000000,0.311885,0.025294,0.0,0.000000,364.726570,366.517068,18.499967
1,TARA_A100001195,244.693184,47.533315,1.320359,4.289834,172.088140,2.515449,0.000000,0.154984,0.283812,0.145679,0.0,0.000000,213.947052,166.578592,5.367209
2,TARA_A200000369,853.130630,278.773744,0.131725,34.702815,613.072158,1.283065,0.013906,0.000000,0.000000,0.162843,0.0,0.000000,1287.972317,614.008761,6.169218
3,TARA_B100000071,390.981233,34.887533,6.978716,9.784341,492.803384,47.468713,33.092871,4.998695,12.207656,2.788364,0.0,0.000000,257.873242,820.968873,72.789015
4,TARA_B100000101,294.029736,4.076444,1.511740,2.432485,754.657715,66.855254,11.205586,1.798751,9.611821,6.269556,0.0,0.000000,295.058737,1153.076761,71.351933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,TARA_B110000875,487.516661,480.559023,0.000000,3.187429,130.620525,0.823546,0.014481,0.892360,0.350864,0.376607,0.0,0.183422,975.542492,237.141417,0.279004
65,TARA_B110000901,582.556942,491.823681,0.045022,17.514660,105.736039,0.812862,0.000000,0.179355,0.083844,0.000000,0.0,0.060794,675.904805,248.282094,0.481363
66,TARA_B110000966,457.376889,300.460263,0.014930,21.543864,123.807868,3.769485,0.000000,4.550520,0.000000,0.000000,0.0,5.698555,415.197262,195.829084,0.388343
67,TARA_B110000969,681.772435,536.995764,0.000000,19.885612,216.116577,0.170541,0.000000,0.000000,0.044197,0.000000,0.0,0.000000,779.299048,271.058567,1.394100


In [3]:
bayinfo = pd.read_csv(r"/Users/theogoodchild-michelman/MIT Dropbox/Izzy Goodchild-Michelman/20,440/caricao_bay_dataset/Cariaco_Supplementary_Data_2023.csv",index_col="sample_name_used_for_this_study")

bayinfo  = bayinfo[bayinfo["ncbi_metatranscriptome_name"].notna()]

bayinfo

,ncbi_metagenome_name,ncbi_metatranscriptome_name,depth,size_fraction,time_point,replicate,sample_type,year,redox_regime,O2,NO3,H2S
sample_name_used_for_this_study,,,,,,,,,,,,
NFL247_1,NaN,R2a247B,247,free-living (0.2-2.7 um),November,1,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
NFL247_2,NaN,R2b247B,247,free-living (0.2-2.7 um),November,2,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
NFL267_1,NaN,R2a267B,267,free-living (0.2-2.7 um),November,1,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
NFL267_2,NaN,R2b267B,267,free-living (0.2-2.7 um),November,2,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
MFL295_1,NaN,R3a295B,295,free-living (0.2-2.7 um),May,1,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
MFL295_2,NaN,R3b295B,295,free-living (0.2-2.7 um),May,2,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
MFL314_1,NaN,R3a314B,314,free-living (0.2-2.7 um),May,1,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
MFL314_2,NaN,R3b314B,314,free-living (0.2-2.7 um),May,2,metatranscriptome,2014,shallow anoxic,0.0,NaN,NaN
NFL900_1,NaN,R2a900BN,900,free-living (0.2-2.7 um),November,1,metatranscriptome,2014,euxinic,0.0,NaN,NaN


In [4]:
meta_cols = bayinfo[["O2", "depth","NO3", "size_fraction"]]

# do a left-join
caricao_sum = (
    caricao_sum
    .merge(meta_cols,
           left_on="sample",
           right_index=True,
           how="left")
)
caricao_sum["Latitude"] = 10.6167
caricao_sum["Longitude"] = -65.1667
caricao_sum["study"] = 0

In [5]:
#for tara
metadata = pd.read_csv(r"/Users/theogoodchild-michelman/MIT Dropbox/Izzy Goodchild-Michelman/Dropbox_files/20.440/Salazar_et_al_2019_sample_info.csv",index_col=0)


meta_cols = metadata[["Oxygen", "Depth.nominal","Latitude","Longitude", "Lyapunov", "NO2","NO3","NO2NO3","PO4", "upper.size.fraction","Temperature","Carbon.total","CO3","HCO3","Density","Alkalinity.total","Nitracline","Brunt.Väisälä","Iron.5m","Residence.time","Okubo.Weiss"]]


# do a left-join
tara_sum = (
    tara_sum
    .merge(meta_cols,
           left_on="sample",
           right_index=True,
           how="left")
)
tara_sum["study"] = 1

In [6]:
tara_sum.columns

Index(['sample', 'cox1', 'cox3', 'dioxygenase', 'dsr', 'katG', 'nap', 'narG',
       'nir', 'nod', 'nos', 'nrd', 'pmoA', 'ribo_reductase', 'sod',
       'Cyt_bd_oxid', 'Oxygen', 'Depth.nominal', 'Latitude', 'Longitude',
       'Lyapunov', 'NO2', 'NO3', 'NO2NO3', 'PO4', 'upper.size.fraction',
       'Temperature', 'Carbon.total', 'CO3', 'HCO3', 'Density',
       'Alkalinity.total', 'Nitracline', 'Brunt.Väisälä', 'Iron.5m',
       'Residence.time', 'Okubo.Weiss', 'study'],
      dtype='object')

In [7]:
tara_sum.columns = ['sample', 'cox1', 'cox3', 'dioxygenase', 'dsr', 'katG', 'nap', 'narG',
       'nir', 'nod', 'nos', 'nrd', 'pmoA', 'ribo_reductase', 'sod',
       'Cyt_bd_oxid','O2', 'depth', 'Latitude', 'Longitude',
       'Lyapunov', 'NO2', 'NO3', 'NO2NO3', 'PO4', 'size_fraction',
       'Temperature', 'Carbon.total', 'CO3', 'HCO3', 'Density',
       'Alkalinity.total', 'Nitracline', 'Brunt.Väisälä', 'Iron.5m',
       'Residence.time', 'Okubo.Weiss','study']


In [8]:
df_all = pd.concat([tara_sum, caricao_sum], ignore_index=True, sort=False)
# df_all.fillna(0,inplace=True)


In [9]:
len(caricao_sum)

48

In [10]:
sizes_scores  = []
for i in df_all["size_fraction"]:
    if i == 1.6:
        sizes_scores.append(0)
    if i == 'free-living (0.2-2.7 um)':
        sizes_scores.append(0)
    if i == 3:
        sizes_scores.append(1)
    if i == 'particle-associated (>2.7 um)':
        sizes_scores.append(2)


In [11]:
df_all["size_scores"] = sizes_scores

In [12]:
df_all.columns

Index(['sample', 'cox1', 'cox3', 'dioxygenase', 'dsr', 'katG', 'nap', 'narG',
       'nir', 'nod', 'nos', 'nrd', 'pmoA', 'ribo_reductase', 'sod',
       'Cyt_bd_oxid', 'O2', 'depth', 'Latitude', 'Longitude', 'Lyapunov',
       'NO2', 'NO3', 'NO2NO3', 'PO4', 'size_fraction', 'Temperature',
       'Carbon.total', 'CO3', 'HCO3', 'Density', 'Alkalinity.total',
       'Nitracline', 'Brunt.Väisälä', 'Iron.5m', 'Residence.time',
       'Okubo.Weiss', 'study', 'mcrA', 'nor', 'size_scores'],
      dtype='object')

In [14]:
df_all = df_all.sort_values("nod",ascending=False)


In [15]:


# ------------------------------------------------------------------
# 1) Read just the first 40 columns (0–39) of your WOA23 CSV:
woa = pd.read_csv(
    "/Users/theogoodchild-michelman/Downloads/woa23_all_o00mn01.csv",
    comment="#",        # skip header comments
    header=None,
    usecols=list(range(40))
)

# rename cols 0–39 to Latitude, Longitude, depths…
depths = ["0","5","10","15","20","25","30","35","40","45",
          "50","55","60","65","70","75","80","85","90","95",
          "100","125","150","175","200","225","250","275","300",
          "325","350","375","400","425","450","475","500","550"]
woa.columns = ["Latitude","Longitude"] + depths

# 2) Grab the 300 m O₂
woa_300 = woa[["Latitude","Longitude","300"]].rename(columns={"300":"O2_300m"}).dropna(subset=["O2_300m"])        # ← skip all missing‐data grid cells


# 3) Your grouped nod data
df_grouped = (
    df_all
    .groupby(["Latitude","Longitude"], as_index=False)
    .agg(nod_expression=("nod","median"))
)
df_grouped = df_grouped.sort_values("nod_expression", ascending=False)


blues_r = px.colors.sequential.Blues_r
custom_blues_r = [
    [0.00, blues_r[0]],   # darkest blue at the very bottom
#     [0.03, blues_r[0]],   # hold that darkest blue until 3%
    [0.7, blues_r[-1]],   # hold that darkest blue until 3%

    [1.00, blues_r[-1]],  # then fade (really jump) to white by 100%
]

# likewise, make “YlOrRd” start in orange instead of yellow:
ylorr = px.colors.sequential.YlOrRd
custom_ylorr = [
    [0.00, ylorr[3]],    # pick the 4th entry (a nice orange) at 0%
    [1.00, ylorr[-1]],   # darkest red at 100%
]

# ------------------------------------------------------------------
# 4) Build the figure and apply tweaks
fig = go.Figure()

# 4a) 300 m oxygen squares
fig.add_trace(go.Scattergeo(
    lon = woa_300["Longitude"],
    lat = woa_300["Latitude"],
    mode = "markers",
    marker = dict(
        symbol     = "square",
        size       = 6,
        color      = woa_300["O2_300m"],
#         colorscale = "RdBu_r",
        colorscale = custom_blues_r,

        cmin       = woa_300["O2_300m"].min(),
        cmax       = woa_300["O2_300m"].max(),
        opacity    = 0.3,
        colorbar   = dict(
            title="O₂ at 300 m",
            x=-0.2,           # push this color-bar to the left
            y=0.5,
            len=0.7
        )
    ),
    name=" "
))

# 4b) nod‐expression bubbles — size them by raw value
max_nod = df_grouped["nod_expression"].max()
fig.add_trace(go.Scattergeo(
    lon = df_grouped["Longitude"],
    lat = df_grouped["Latitude"],
    mode = "markers",
    marker = dict(
        size      = df_grouped["nod_expression"],
        sizemode  = "area",
        # sizeref formula so that the largest bubble ≃40 px diameter:
        sizeref   = 2.*max_nod/(40.**2),
        sizemin   = 4.5,
        color     = df_grouped["nod_expression"],
        colorscale= custom_ylorr,
        opacity   = 0.8,
        colorbar  = dict(
            title="nod expression",
            x=1.1,           # push this color-bar to the right
            y=0.5,
            len=0.7
        )
    ),
    name=" "
))

# ------------------------------------------------------------------
# 5) Whiten out everything except your markers
fig.update_layout(
    title="nod expression and global oxygen minimum zones",
    geo = dict(
        projection_type="natural earth",
        
        # draw land & ocean as white instead of grey
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="white",
        
#         # restore boundaries
#         showcontinents=True,
#         countrycolor="black",
        showcoastlines=True,
        coastlinecolor="black",
        
        # get rid of the extra GeoJSON frame
        showframe=False,
        
        # increase resolution if you want smoother borders
        resolution=50
    )
)


fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",

)

fig.show()


NameError: name 'px' is not defined

In [16]:

# assume df_all has columns: Latitude, Longitude, nod

# 1) Group by coordinate, summing the nod expression
df_grouped = (
    df_all
    .groupby(["Latitude", "Longitude"], as_index=False)
    .agg(nod_expression=("nod", "median"))
)

df_grouped = df_grouped.sort_values("nod_expression",ascending=False)
# 2) Make the map
fig = px.scatter_geo(
    df_grouped,
    lat="Latitude",
    lon="Longitude",
    si    [0.03, blues_r[0]],   # hold that darkest blue until 3%
ze_max=40,               # ↑ make the largest bubbles up to 100px diameter
    opacity=0.65,    
    size="nod_expression",      # bubble area ~ sum of nod expression
    color="nod_expression",     # color scale ~ sum of nod expression
    projection="natural earth",
    title="Geographic distribution of summed nod expression",
    hover_name="nod_expression",
    hover_data={"Latitude": True, "Longitude": True, "nod_expression": True}
)

# 3) Tweak marker size
fig.update_traces(marker=dict(sizemode="area", sizemin=4.5))

fig.show()


SyntaxError: positional argument follows keyword argument (1621950544.py, line 27)

In [ ]:
# metadata = pd.read_csv(r"/Users/theogoodchild-michelman/MIT Dropbox/Izzy Goodchild-Michelman/Dropbox_files/20.440/Salazar_et_al_2019_sample_info.csv",index_col=0)

# metadata

# metasub = metadata.loc[list(scores.index)]
# metasub

In [ ]:
# markers = pd.read_csv("/Users/theogoodchild-michelman/Downloads/all_tara_counts_sum.csv",index_col=0)
# # markers = markers.iloc[1:]

# markers.index = [i.strip() for i in markers.index]
# markers.head()

In [ ]:


# your marker lists
# aerobic_markers = ["cox3","cox1"]
aerobic_markers = [
    "cox1",
    "cox3",
    "katG",
    "ribo_reductase"]
#     "Cyt_bd_oxid"]

anaerobic_markers = ["nap",
    "dsr",
    "nir",
    "nos",
    "nrd"]

# anaerobic_markers = ["nir"]

def compute_score(row, markers):
    # sum raw counts across the markers
    raw_sum = row[markers].sum()
    # how many of those markers are actually present (>0)
    n_found = (row[markers] > 0).sum()
    # log2(sum + 1) * number found
#     return np.log2(raw_sum + 1)
    return raw_sum


def score_geomean_frac(row, markers):
    counts = row[markers]
    # 1) geometric mean of (counts+1), then subtract 1
    gm = np.exp(np.log(counts + 1).mean()) - 1
    # 2) fraction of markers detected
    frac_found = (counts > 0).sum() / len(markers)
    return gm * frac_found


scores = pd.DataFrame({
    "aerobic_score": df_all.apply(lambda r: compute_score(r, aerobic_markers), axis=1),
    "anaerobic_score": df_all.apply(lambda r: compute_score(r, anaerobic_markers), axis=1),
})

# scores is indexed by sample
scores


In [ ]:
df_all["aerobic_score"] = scores["aerobic_score"]
df_all["anaerobic_score"] = scores["anaerobic_score"]
df_all

In [ ]:
df_all.to_csv("alltara_caricao_TPM_counts.csv")
[float(i) for i in df_all["nod"]]

In [ ]:
nods = [float(i) for i in df_all["nod"]]
nonzeronods = [i for i in nods if i > 0 ]
np.percentile(nonzeronods, [25, 50, 75])
plt.hist(nonzeronods,bins=30)
plt.axvline(0.44, color="red")


In [ ]:

# Create a color vector based on nod > 0
colors = df_all["nod"].apply(lambda x: "red" if x > 0.41  else "blue")

# Make scatter plot
plt.scatter(df_all["aerobic_score"], df_all["O2"],c=colors)

plt.xlabel("aerobic_score")
plt.ylabel("O2")
plt.title("aerobic_score vs O2 level red = n0d presence ")
plt.show()

In [ ]:
plt.figure(figsize=(9,7))

# Create a color vector based on nod > 0
colors = df_all["nod"].apply(lambda x: "red" if x > 0.41  else "blue")

# Make scatter plot
plt.scatter(df_all["anaerobic_score"], df_all["O2"],c=colors)

plt.xlabel("anaerobic_score")
plt.ylabel("O2")
plt.title("anaerobic_score vs O2 ")
plt.show()

In [ ]:
# Make scatter plot
plt.scatter(df_all["anaerobic_score"], df_all["O2"],c=colors)

plt.xlabel("anaerobic_score")
plt.ylabel("depth")
plt.title(" ")
plt.show()

In [ ]:


# Data
x = df_all["nod"]
y = df_all["size_scores"]
size_scores = df_all["size_scores"]   # values 0, 1, or 2

# Define one color per category
color_map = {0: "blue", 1: "orange", 2: "green"}
colors = size_scores.map(color_map)

# Plot
fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(x, y, c=colors, alpha=0.7, s=80)

# Add the anoxic cutoff line
# cutoff_line = ax.axhline(20, color="red")

# Build legend handles for size_score
score_handles = [
    mpatches.Patch(color=col, label=f"particle score= {score}")
    for score, col in color_map.items()
]

# Add the cutoff line handle to the list
# (you can also use `cutoff_line` directly instead of creating a new Patch)
score_handles.append(cutoff_line)

# Draw the legend
ax.legend(handles=score_handles, loc="upper right", title="Legend")

# Labels & title
ax.set_xlabel("nod gene expression detected in sample (TPM)")
ax.set_ylabel("size_scores")
ax.set_title("nod gene expression vs Oxygen")

plt.tight_layout()
plt.show()


In [ ]:
o2 = 14000
# Spearman correlation: nod vs aerobic_score
rho_aerobic, pval_aerobic = pearsonr(df_all[df_all["size_scores"] != 1]["size_scores"], df_all[df_all["size_scores"] != 1]['nod'])
print(f"Spearman correlation (nod vs aerobic_score): rho={rho_aerobic:.3f}, p-value={pval_aerobic:.3g}")


In [ ]:
rho_aerobic, pval_aerobic = pearsonr(df_all["size_scores"], df_all['nod'])
print(f"Spearman correlation (nod vs aerobic_score): rho={rho_aerobic:.3f}, p-value={pval_aerobic:.3g}")


In [ ]:

plt.rcParams.update({'font.size': 23})


# Extract the x and y values.
x = df_all["aerobic_score"]
y = df_all["O2"]


# colors = df_all["nod"].apply(lambda x: "red" if x > 0.44  else "blue")

# Compute linear regression coefficients: slope and intercept.
slope, intercept = np.polyfit(x, y, 1)
# Evaluate the fitted line.
y_pred = slope * x + intercept
# Compute the R² score.
r_squared = r2_score(y, y_pred)
plt.figure(figsize=(10, 7))

# Plot the scatter plot.
plt.scatter(x, y, alpha=0.4,s=100)


# Add labels and a title.
plt.ylabel("O2 (μM)")
plt.xlabel("aerobic score (TPM)")

plt.title(" ")

plt.rcParams.update({'font.size': 16})
plt.axhline(20, color="red",label="anoxic cutoff")

# plt.axhspan(-700, -100, color="grey", alpha=0.25, label="O2 minimum zone depth")

plt.legend(loc="upper right")


In [ ]:
# Make scatter plot

plt.rcParams.update({'font.size': 20})


# Extract the x and y values.
y = df_all["nod"]
x = df_all["anaerobic_score"]


# colors = df_all["nod"].apply(lambda x: "red" if x > 0.44  else "blue")

# Compute linear regression coefficients: slope and intercept.
slope, intercept = np.polyfit(x, y, 1)
# Evaluate the fitted line.
y_pred = slope * x + intercept
# Compute the R² score.
r_squared = r2_score(y, y_pred)
plt.figure(figsize=(9, 7))

# Plot the scatter plot.
plt.scatter(x, y, label="water samples",alpha=0.4,s=80)


# Add labels and a title.
plt.xlabel("anaerobic_score (TPM)")
plt.ylabel("nod expression (TPM)")
# plt.axhline(20, color="red",label="anoxic cutoff")
plt.title(" ")
plt.legend(loc="upper right")




In [ ]:
df_all["NO3"]

In [ ]:
# Make scatter plot
colors = df_all["nod"].apply(lambda x: "red" if x > 0.41  else "blue")

plt.scatter(df_all["NO3"], df_all["nod"],c=colors)

plt.xlabel("NO3")
plt.ylabel("nod")
plt.title(" ")
plt.show()

In [ ]:

o2 = 14000
# Spearman correlation: nod vs aerobic_score
rho_aerobic, pval_aerobic = pearsonr(df_all[df_all["O2"] < 20]["aerobic_score"], df_all[df_all["O2"] < 20]['nod'])

# rho_aerobic, pval_aerobic = pearsonr(df_all["anaerobic_score"], df_all['nod'])


# Print results
print(f"Spearman correlation (nod vs aerobic_score): rho={rho_aerobic:.3f}, p-value={pval_aerobic:.3g}")

